In [89]:
#Librerias
import pandas as pd
import mysql.connector as connection
import pyodbc
import warnings

warnings.filterwarnings('ignore')

In [90]:
#Accesos MySQL
host="10.1.1.198"
database ="db_sidifoca"
user="jcoronado"
passwd="#Minedu2021#"

#Accesos a SQLServer
host1="10.200.9.11"
database1 ='db_sysdifoca'
user1="user_sysdifoca"
passwd1="user_sysdifoca"

In [91]:
def sql_update_table(tabla, llave):
    #traer tabla MySQL (origen)
    try:
        con = connection.connect(host=host, database =database,user=user, passwd=passwd)
        query = "select * from "+tabla
        dfx = pd.read_sql(query,con)
        con.close() #close the connection
    except Exception as e:
        con.close()
        print(str(e))
    
    dfx=dfx.replace("'", "", regex=True)
        
    #traer tabla SQLServer (destino)
    try:
        con = pyodbc.connect('DRIVER={SQL Server};SERVER='+host1+';DATABASE='+database1+';UID='+user1+';PWD='+passwd1)
        query = "select * from "+tabla
        dfy = pd.read_sql(query,con)
        con.close()
    except Exception as e:
        con.close()
        print(str(e))
    
    #buscar nuevos valores
    df = pd.concat([dfy, dfx]) 
    df = df.reset_index(drop=True)
    df["cant"]=1
   
    df1=df.groupby(llave)["cant"].count()
    df2 = pd.DataFrame(df1)
    df2.reset_index(inplace=True)

    df3 = df.merge(df2, on=llave, how="left")
    df4=df3[df3["cant_y"]==1]
    df4.pop("cant_x")
    df4.pop("cant_y")
    df4 = df4.astype (str)
    
    df4.reset_index(inplace=True)
    df4.pop("index")
    
    cols = ",".join([str(i) for i in df4.columns.tolist()])
    con = pyodbc.connect('DRIVER={SQL Server};SERVER='+host1+';DATABASE='+database1+';UID='+user1+';PWD='+passwd1)
    cur=con.cursor()
    for (index,rs) in df4.iterrows():
        vals = "','".join([str(xi) for xi in df4.values.tolist()[index]])
        sql = "INSERT INTO "+tabla+" (" +cols + ") VALUES ('" + vals + "')"
        cur.execute(sql)
    con.commit()
    con.close()      

In [92]:
def sql_create_table (tabla_sql):
    #traer tabla
    try:
        con = connection.connect(host=host, database =database,user=user, passwd=passwd)
        query = "select * from "+tabla_sql
        dfx = pd.read_sql(query,con)
        con.close() #close the connection
    except Exception as e:
        con.close()
        print(str(e))
    
    columnName = list(dfx.columns.values)
    dataList = []
    for x in dfx.dtypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('text')
    
    #crear script de crear tabla
    createTable = 'CREATE TABLE '+tabla_sql+' ('
    for i in range(len(dataList)):
        createTable = createTable + columnName[i] + ' ' + dataList[i] + ','
    createTable = createTable[:-1] + ' );'
   
    con = pyodbc.connect('DRIVER={SQL Server};SERVER='+host1+';DATABASE='+database1+';UID='+user1+';PWD='+passwd1)
    cur=con.cursor()
    cur.execute(createTable)
    con.commit()
    con.close()

In [95]:
#Aplicar funcion crear tablas (tabla_sql)
sql_create_table("T_GENM_USUARIO")

In [96]:
#Aplicar funcion (tabla_sql, columna_llave)
sql_update_table("T_GENM_USUARIO","id_usuario")

